# Robust Knapsack

In [1]:
import cvxpy as cp
import numpy as np
import torch
import matplotlib.pyplot as plt

import lropt
from lropt.parameter import Parameter
from lropt.robust_problem import RobustProblem
from lropt.uncertain import UncertainParameter

np.random.seed(seed=1234)

In [2]:
N = 50
b = 2000

c = 2*np.random.randint(low=5, high=10, size=N)    # profit coefficients
w = 2*np.random.randint(low=10, high=41, size=N)   # nominal weights
delta = np.diag(0.2*w)                             # maximum deviations
r = 2                                              # uncertainty budget

In [3]:
uncertainty_set = lropt.Budget(rho1=1, rho2=r)
x = cp.Variable(N, boolean=True)
z = UncertainParameter(N, uncertainty_set=uncertainty_set)

objective = cp.Maximize(c@x)
constraints = [
                (w + delta@z)@x <= b,
              ]
prob = RobustProblem(objective=objective, constraints=constraints)

In [4]:
prob.solve()

626.0